In [147]:
PINECONE = {
    "API_KEY": "pcsk_2b78HU_TqDQRdqmBRhgvc6RFtLamaqMW6NTqq7kziXPLcZEa4aabk5iTSwraqVCx3dF6iy",
    "ENVIRONMENT": "us-west1-gcp"
}

In [ ]:
!pip install pinecone-client sentence-transformers transformers odfpy gradio
!pip install "pinecone-client[grpc]"

In [ ]:
!pip install odfpy

In [150]:
!mkdir -p docs_odt  # Crear carpeta si no existe
!mv isaias_cv.odt  octavio_cv.odt santiago_cv.odt docs_odt/

In [151]:
import os
import odf.opendocument
import odf.text
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from transformers import pipeline
import numpy as np
import random
import gradio as gr
import json

In [170]:
# Configuración de Pinecone (reemplaza con tus credenciales)
PINECONE = {
    "API_KEY": "pcsk_2b78HU_TqDQRdqmBRhgvc6RFtLamaqMW6NTqq7kziXPLcZEa4aabk5iTSwraqVCx3dF6iy",  # Usa tu propia API key de Pinecone
    "ENVIRONMENT": "us-west1-gcp"  # Cambia según tu región
}

# Inicializar el cliente de Pinecone
pinecone_client = Pinecone(api_key=PINECONE["API_KEY"], environment=PINECONE["ENVIRONMENT"])

# Crear o acceder al índice en Pinecone
index_name = "rag-index"
embedding_dimension = 384  # Asegúrate de que sea compatible con tu modelo de embeddings

# Verificar si el índice existe, si no, lo crea
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [ ]:
# Cargar y leer el contenido del archivo .odt
def leer_documentos_odt(ruta_archivo):
    documento = odf.opendocument.load(ruta_archivo)
    contenido = []
    for elemento in documento.getElementsByType(odf.text.P):
        # Extraer el texto de cada elemento <P> de forma segura
        # Cambiado de odf.text.Text a odf.element.Text
        texto_parrafo = ''.join([node.data for node in elemento.childNodes if isinstance(node, odf.element.Text)])
        if texto_parrafo.strip():  # Asegurarse de que el texto no esté vacío
            contenido.append(texto_parrafo.strip())
    return "\n".join(contenido)

# Ruta del directorio que contiene los archivos .odt
ruta_directorio = "/content/docs_odt"

# Iterar sobre los archivos en el directorio
for nombre_archivo in os.listdir(ruta_directorio):
    # Verificar si el archivo es un archivo .odt
    if nombre_archivo.endswith(".odt"):
        # Construir la ruta completa del archivo
        ruta_archivo = os.path.join(ruta_directorio, nombre_archivo)

        # Leer el archivo y mostrar los primeros 100 caracteres
        contenido_documento = leer_documentos_odt(ruta_archivo)
        print(f"Contenido del documento {nombre_archivo}: {contenido_documento[:100]}")

In [172]:
# Normalización de vectores usando Z-score
def normalizar_vector_zscore(vector):
    """Normaliza un vector utilizando Z-score."""
    vector = np.array(vector)  # Convertir a NumPy array
    mean = np.mean(vector)
    std = np.std(vector)
    if std == 0:  # Manejar el caso de desviación estándar 0
        return vector
    else:
        return (vector - mean) / std

In [ ]:
# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embedding del contenido del documento
embedding = model.encode(contenido_documento)

print(f"Dimensión del embedding: {len(embedding)}")

# Iterar sobre los archivos en el directorio
for nombre_archivo in os.listdir(ruta_directorio):
    if nombre_archivo.endswith(".odt"):
        ruta_archivo = os.path.join(ruta_directorio, nombre_archivo)

        # Leer el archivo
        contenido_documento = leer_documentos_odt(ruta_archivo)

        # Generar embedding del contenido del documento
        embedding = model.encode(contenido_documento)
        # ===>>> Add Normalization here <<<===
        embedding = normalizar_vector_zscore(embedding).tolist()
        # ===>>> End Normalization <<<===

        # Crear un diccionario con el embedding y los metadatos
        documento_id = nombre_archivo  # Usar el nombre del archivo como ID
        metadata = {"text": contenido_documento}

        # Subir el embedding a Pinecone
        index = pinecone_client.Index(index_name)
        index.upsert(vectors=[{
            "id": documento_id,
            "values": embedding,
            "metadata": metadata
        }])

        print(f"Documento '{documento_id}' subido a Pinecone.")

# Verificar estadísticas del índice
stats = index.describe_index_stats()
print("Estadísticas del índice:", stats)

In [174]:
# Función para consultar Pinecone y obtener documentos relevantes
def consultar_pinecone(query, top_k=3):
    # Generar el embedding de la consulta
    query_embedding = model.encode(query).tolist()

    # Normalizar el vector de consulta
    query_embedding = normalizar_vector_zscore(query_embedding).tolist()

    # Realizar la consulta en Pinecone
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_values=True,
        include_metadata=True,
        namespace=""
    )

    return results.matches


In [189]:
# Cargar el modelo de generación de texto
#generator = pipeline("text2text-generation", model="google/flan-t5-small")
generator = pipeline("text2text-generation", model="google/flan-t5-base")
# Función para generar la respuesta utilizando contexto recuperado
def generar_respuesta(query, contexto):
    if not contexto.strip():
        return "No se encontraron documentos relevantes para generar una respuesta."

    prompt = f"Contexto:\n{contexto}\n\nPregunta: {query}\nRespuesta:"

    try:
        # Generar varias respuestas con temperatura y longitud máxima ajustadas
        respuestas = generator(
            prompt,
            max_length=30,
            num_beams=3,# Ajusta la longitud máxima según tus necesidades
            num_return_sequences=3,  # Genera 3 respuestas diferentes
            temperature=0.7  # Ajusta la temperatura para controlar la diversidad
        )

        # Seleccionar una respuesta aleatoria de las generadas
        mejor_respuesta = random.choice(respuestas)['generated_text']
        return mejor_respuesta
    except Exception as e:
        return f"Error al generar la respuesta: {str(e)}"


In [190]:
# Función principal del chatbot RAG
def chatbot_rag(query):
    try:
        # Consultar Pinecone y obtener los resultados
        resultados = consultar_pinecone(query, top_k=3)

        if not resultados:
            return "No se encontraron documentos relevantes para esta consulta."

        # Extraer el texto del contexto de los metadatos
        contexto = " ".join([res.metadata['text'] for res in resultados if res.metadata and res.metadata.get('text')])

        # Verifica si se generó un contexto válido
        if not contexto:
            return "No se pudieron construir contextos relevantes a partir de los documentos recuperados."

        # Generar la respuesta usando la función generar_respuesta
        respuesta = generar_respuesta(query, contexto)
        return respuesta

    except Exception as e:
        return f"Error en el flujo RAG: {str(e)}"


In [193]:
iface = gr.Interface(
    fn=chatbot_rag,
    inputs=gr.Textbox(lines=2, placeholder="Ingresa tu consulta aquí..."),
    outputs="text",
    title="Chatbot RAG para CVs",
    description="Consulta información sobre  CVs cargados."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://797bb5e4838980b94f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
